# Use of Nucleus to create a dataset for QCNN
Code modified from [this work](https://google.github.io/deepvariant/posts/2019-01-31-using-nucleus-and-tensorflow-for-dna-sequencing-error-correction/)

In [ ]:

%%capture
!gsutil cp gs://deepvariant/case-study-testdata/NA12878_sliced.bam NA12878_sliced.bam
!gsutil cp gs://deepvariant/case-study-testdata/NA12878_sliced.bam.bai NA12878_sliced.bam.bai

!wget ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/release/NA12878_HG001/NISTv3.3.2/GRCh37/HG001_GRCh37_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_PGandRTGphasetransfer.vcf.gz -O NA12878_calls.vcf.gz
!wget ftp://ftp-trace.ncbi.nlm.nih.gov/giab/ftp/release/NA12878_HG001/NISTv3.3.2/GRCh37/HG001_GRCh37_GIAB_highconf_CG-IllFB-IllGATKHC-Ion-10X-SOLID_CHROM1-X_v.3.3.2_highconf_PGandRTGphasetransfer.vcf.gz.tbi -O NA12878_calls.vcf.gz.tbi

!gsutil cp gs://deepvariant/case-study-testdata/hs37d5.fa.gz hs37d5.fa.gz
!gsutil cp gs://deepvariant/case-study-testdata/hs37d5.fa.gz.fai hs37d5.fa.gz.fai
!gsutil cp gs://deepvariant/case-study-testdata/hs37d5.fa.gz.gzi hs37d5.fa.gz.gzi

# To run this we need pip==0.21.*

In [ ]:

!pip install -q google-nucleus==0.5.6
!pip install -q tensorflow==2.3.0

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import random

import numpy as np

from nucleus.io import fasta
from nucleus.io import sam
from nucleus.io import vcf
from nucleus.io.genomics_writer import TFRecordWriter
from nucleus.protos import reads_pb2
from nucleus.util import cigar
from nucleus.util import ranges
from nucleus.util import utils

# Import TensorFlow after Nucleus.
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
_ALLOWED_CIGAR_OPS = frozenset([cigar.CHAR_TO_CIGAR_OPS[op] for op in 'MX='])
_ALLOWED_BASES = 'ACGT'
_TRAIN = 'train.tfrecord'
_EVAL = 'eval.tfrecord'
_TEST = 'test.tfrecord'

In [ ]:
def generate_tfrecord_datasets(hparams):
  """Writes out TFRecords files for training, evaluation, and test datasets."""
  if not os.path.exists(hparams.out_dir):
    os.makedirs(hparams.out_dir)

  # Fraction of examples in each dataset.
  train_eval_test_split = [0.7, 0.2, 0.1]
  num_train_examples = 0
  num_eval_examples = 0
  num_test_examples = 0

  # Generate training, test, and evaluation examples.
  with TFRecordWriter(os.path.join(hparams.out_dir, _TRAIN)) as train_out, \
       TFRecordWriter(os.path.join(hparams.out_dir, _EVAL)) as eval_out, \
       TFRecordWriter(os.path.join(hparams.out_dir, _TEST)) as test_out:
    all_examples = make_ngs_examples(hparams)
    for example in all_examples:
      r = random.random()
      if r < train_eval_test_split[0]:
        train_out.write(proto=example)
        num_train_examples += 1
      elif r < train_eval_test_split[0] + train_eval_test_split[1]:
        eval_out.write(proto=example)
        num_eval_examples += 1
      else:
        test_out.write(proto=example)
        num_test_examples += 1
  print('# of training examples: %d' % num_train_examples)
  print('# of evaluation examples: %d' % num_eval_examples)
  print('# of test examples: %d' % num_test_examples)


def make_ngs_examples(hparams):
  """Generator function that yields training, evaluation and test examples."""
  ref_reader = fasta.IndexedFastaReader(input_path=hparams.ref_path)
  vcf_reader = vcf.VcfReader(input_path=hparams.vcf_path)
  read_requirements = reads_pb2.ReadRequirements()
  sam_reader = sam.SamReader(
      input_path=hparams.bam_path, read_requirements=read_requirements)

  # Use a separate SAM reader to query for reads falling in the pileup range.
  sam_query_reader = sam.SamReader(
      input_path=hparams.bam_path, read_requirements=read_requirements)
  used_pileup_ranges = set()
  with ref_reader, vcf_reader, sam_reader, sam_query_reader:
    for read in sam_reader:

      # Check that read has cigar string present and allowed alignment.
      if not read.alignment.cigar:
        print('Skipping read, no cigar alignment found')
        continue
      if not has_allowed_alignment(read):
        continue

      # Obtain window that will be used to construct an example.
      read_range = utils.read_range(read)
      ref = ref_reader.query(region=read_range)
      pileup_range = get_pileup_range(hparams, read, read_range, ref)

      # Do not construct multiple examples with the same pileup range.
      pileup_range_serialized = pileup_range.SerializeToString()
      if pileup_range_serialized in used_pileup_ranges:
        continue
      used_pileup_ranges.add(pileup_range_serialized)

      # Get reference sequence, reads, and truth variants for the pileup range.
      pileup_reads = list(sam_query_reader.query(region=pileup_range))
      pileup_ref = ref_reader.query(region=pileup_range)
      pileup_variants = list(vcf_reader.query(region=pileup_range))
      if is_usable_example(pileup_reads, pileup_variants, pileup_ref):
        yield make_example(hparams, pileup_reads, pileup_ref, pileup_range)


def get_pileup_range(hparams, read, read_range, ref):
  """Returns a range that will be used to construct one example."""

  # Find error positions where read and reference differ.
  ngs_read_length = read_range.end - read_range.start
  error_indices = [
      i for i in range(ngs_read_length) if ref[i] != read.aligned_sequence[i]
  ]

  # If read and reference sequence are the same, create an example centered
  # at middle base of read.
  if not error_indices:
    error_idx = ngs_read_length // 2

  # If read and reference differ at one or more positions, create example
  # centered at a random error position.
  else:
    error_idx = random.choice(error_indices)

  error_pos = read_range.start + error_idx
  flank_size = hparams.window_size // 2
  return ranges.make_range(
      chrom=read_range.reference_name,
      start=error_pos - flank_size,
      end=error_pos + flank_size + 1)


def has_allowed_alignment(read):
  """Determines whether a read's CIGAR string has the allowed alignments."""
  return all([c.operation in _ALLOWED_CIGAR_OPS for c in read.alignment.cigar])


def is_usable_example(reads, variants, ref_bases):
  """Determines whether a particular reference region and read can be used."""
  # Discard examples with variants or no mapped reads.
  if variants or not reads:
    return False

  # Use only examples where all reads have simple alignment and allowed bases.
  for read in reads:
    if not has_allowed_alignment(read):
      return False
    if any(base not in _ALLOWED_BASES for base in read.aligned_sequence):
      return False

  # Reference should only contain allowed bases.
  if any(base not in _ALLOWED_BASES for base in ref_bases):
    return False
  return True


def make_example(hparams, pileup_reads, pileup_ref, pileup_range):
  """Takes in an input sequence and outputs tf.train.Example ProtocolMessages.

  Each example contains the following features: A counts, C counts, G counts,
  T counts, reference sequence, correct base label.
  """
  assert len(pileup_ref) == hparams.window_size
  example = tf.train.Example()
  base_counts = np.zeros(shape=[hparams.window_size, len(_ALLOWED_BASES)])

  for read in pileup_reads:
    read_position = read.alignment.position.position
    read_ints = [_ALLOWED_BASES.index(b) for b in read.aligned_sequence]
    one_hot_read = np.zeros((len(read_ints), len(_ALLOWED_BASES)))
    one_hot_read[np.arange(len(one_hot_read)), read_ints] = 1

    window_start = read_position - pileup_range.start
    window_end = window_start + len(read_ints)

    # If read falls outside of window, adjust start/end indices for window.
    window_start = max(0, window_start)
    window_end = min(window_end, hparams.window_size)

    # We consider four possible scenarios for each read and adjust start/end
    # indices to only include portions of read that overlap the window.
    # 1) Read extends past 5' end of window
    # 2) Read extends past 3' end of window
    # 3) Read extends past 5' and 3' ends of window
    # 4) Read falls entirely within window
    if window_start == 0 and window_end != hparams.window_size:
      read_start = pileup_range.start - read_position
      read_end = None
    if window_end == hparams.window_size and window_start != 0:
      read_start = None
      read_end = -1 * ((read_position + len(read_ints)) - pileup_range.end)
    if window_start == 0 and window_end == hparams.window_size:
      read_start = pileup_range.start - read_position
      read_end = read_start + hparams.window_size
    if window_start != 0 and window_end != hparams.window_size:
      read_start = None
      read_end = None
    base_counts[window_start:window_end] += one_hot_read[read_start:read_end]

  # Use fractions at each position instead of raw base counts.
  base_counts /= np.expand_dims(np.sum(base_counts, axis=-1), -1)

  # Save counts/fractions for each base separately.
  features = example.features
  for i in range(len(_ALLOWED_BASES)):
    key = '%s_counts' % _ALLOWED_BASES[i]
    features.feature[key].float_list.value.extend(list(base_counts[:, i]))

  features.feature['ref_sequence'].int64_list.value.extend(
      [_ALLOWED_BASES.index(base) for base in pileup_ref])
  flank_size = hparams.window_size // 2
  true_base = pileup_ref[flank_size]
  features.feature['label'].int64_list.value.append(
      _ALLOWED_BASES.index(true_base))

  return example

In [ ]:
def get_dataset(hparams, filename, num_epochs):
  """Reads in and processes the TFRecords dataset.

  Builds a pipeline that returns pairs of features, label.
  """

  # Define field names, types, and sizes for TFRecords.
  proto_features = {
      'A_counts':
          tf.io.FixedLenFeature(shape=[hparams.window_size], dtype=tf.float32),
      'C_counts':
          tf.io.FixedLenFeature(shape=[hparams.window_size], dtype=tf.float32),
      'G_counts':
          tf.io.FixedLenFeature(shape=[hparams.window_size], dtype=tf.float32),
      'T_counts':
          tf.io.FixedLenFeature(shape=[hparams.window_size], dtype=tf.float32),
      'ref_sequence':
          tf.io.FixedLenFeature(shape=[hparams.window_size], dtype=tf.int64),
      'label':
          tf.io.FixedLenFeature(shape=[1], dtype=tf.int64),
  }

  def _process_input(proto_string):
    """Helper function for input function that parses a serialized example."""

    parsed_features = tf.io.parse_single_example(
        serialized=proto_string, features=proto_features)

    # Stack counts/fractions for all bases to create input of dimensions
    # `hparams.window_size` x len(_ALLOWED_BASES).
    feature_columns = []
    for base in _ALLOWED_BASES:
      feature_columns.append(parsed_features['%s_counts' % base])
    features = tf.stack(feature_columns, axis=-1)
    label = parsed_features['label']
    return features, label

  ds = tf.data.TFRecordDataset(filenames=filename)
  ds = ds.map(map_func=_process_input)
  ds = ds.shuffle(buffer_size=10000, reshuffle_each_iteration=True)
  ds = ds.batch(batch_size=hparams.batch_size).repeat(count=num_epochs)
  return ds

In [ ]:
class BaseHparams(object):
  """Default hyperparameters."""

  def __init__(self,
               total_epochs=100,
               learning_rate=0.004,
               l2=0.001,
               batch_size=256,
               window_size=21,
               ref_path='hs37d5.fa.gz',
               vcf_path='NA12878_calls.vcf.gz',
               bam_path='NA12878_sliced.bam',
               out_dir='examples',
               model_dir='ngs_model',
               log_dir='logs'):

    self.total_epochs = total_epochs
    self.learning_rate = learning_rate
    self.l2 = l2
    self.batch_size = batch_size
    self.window_size = window_size
    self.ref_path = ref_path
    self.vcf_path = vcf_path
    self.bam_path = bam_path
    self.out_dir = out_dir
    self.model_dir = model_dir
    self.log_dir = log_dir

In [ ]:
hparams = BaseHparams()

train_dataset = get_dataset(
      hparams, filename=os.path.join(hparams.out_dir, _TRAIN), num_epochs=1)
  eval_dataset = get_dataset(
      hparams, filename=os.path.join(hparams.out_dir, _EVAL), num_epochs=1)
  test_dataset = get_dataset(
      hparams, filename=os.path.join(hparams.out_dir, _TEST), num_epochs=1)